In [61]:
!pip install pandas matplotlib mplfinance

In [69]:
import pandas as pd
import numpy as np
import glob
import os

from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf


## DATA CLEANING

In [ ]:
folder_path = 'Laporan_keuangan'

all_files = glob.glob(os.path.join(folder_path, "*.csv"))
df_list = []
ratios_recap = pd.DataFrame()
for idx, file in enumerate(all_files):
    try:
        print(file)
        df = pd.read_csv(file)
        # break
        df.replace('--', np.nan, inplace=True)
        missing_percentage = df.isnull().mean(axis=1)
        threshold = 0.3
        df = df[missing_percentage <= threshold]
        columns_to_convert = df.columns.difference(['Date'])
        for col in columns_to_convert:
            if df[col].dtype == 'object':  # Check if the column is string type
                df[col] = df[col].str.replace(',', '')
        df[columns_to_convert] = df[columns_to_convert].astype(np.float64)
        df = df.apply(lambda col: col.fillna(col.mean()) if col.dtype in [np.float64, np.int64] else col)
        df['Date'] = df['Date'].apply(lambda x: '09/15/2024' if 'TTM' in str(x) else x)
        df['Date'] = pd.to_datetime(df['Date'],format='%m/%d/%Y')
        financial_metrics = df[[
            'Total Revenue',
            'Net Income Common Stockholders',
            'Total Equity Gross Minority Interest',
            'Total Debt',
            'EBITDA',
            'Free Cash Flow'
        ]]
        financial_metrics = financial_metrics.rename(columns={
            'Total Revenue': 'Total_Revenue',
            'Net Income Common Stockholders': 'Net_Income',
            'Total Equity Gross Minority Interest': 'Total_Equity',
            'Total Debt': 'Total_Debt',
            'Free Cash Flow': 'Free_Cash_Flow'
        })
        scaler = MinMaxScaler()
        ratios = {}
        ratios['Company'] = file.split('\\')[-1].split('_')[0]
        ratios['Net_Profit_Margin'] = (financial_metrics['Net_Income'] / financial_metrics['Total_Revenue']).mean()
        ratios['ROE'] = (financial_metrics['Net_Income'] / financial_metrics['Total_Equity']).mean()
        ratios['Debt_to_Equity'] = (financial_metrics['Total_Debt'] / financial_metrics['Total_Equity']).mean()
        ratios['EBITDA_Margin'] = (financial_metrics['EBITDA'] / financial_metrics['Total_Revenue']).mean()
        ratios['free_cash_flow'] = (financial_metrics['Free_Cash_Flow'].mean())
        df_ratio=pd.DataFrame([ratios])
        ratios_recap = pd.concat([ratios_recap, df_ratio], ignore_index=True)
        print(ratios_recap)
    except Exception as e:
        print(f"Error processing file {file}: {e}")
    # if idx ==2:break
ratios_recap.to_csv('combined_finance_ratio.csv', index=False)
# df_list.to_csv('combined.csv', index=False)

In [55]:
scaler = MinMaxScaler()
df_ratio = pd.read_csv('combined_finance_ratio.csv')
df_ratio['Debt_to_Equity_Score'] = 1 - scaler.fit_transform(df_ratio[['Debt_to_Equity']])

scoring_metrics = ['Net_Profit_Margin', 'ROE', 'EBITDA_Margin', 'free_cash_flow']

normalized_scores = scaler.fit_transform(df_ratio[scoring_metrics])

for i, metric in enumerate(scoring_metrics):
    df_ratio[metric + '_Score'] = normalized_scores[:, i]
df_ratio['Total_Score'] = (
    df_ratio['Net_Profit_Margin_Score'] * 0.25 +
    df_ratio['ROE_Score'] * 0.25 +
    df_ratio['EBITDA_Margin_Score'] * 0.25 +
    df_ratio['free_cash_flow_Score'] * 0.15 +
    df_ratio['Debt_to_Equity_Score'] * 0.10
)

top_20 = df_ratio.sort_values(by='Total_Score', ascending=False).head(20)

top_20.to_csv('top_20_companies_for_investment.csv', index=False)


In [56]:
df_ratio.to_csv('combined_finance_ratio_2.csv', index=False)

In [57]:
df_ratio.sort_values(by='Total_Score', ascending=False).head(20)

,Company,Net_Profit_Margin,ROE,Debt_to_Equity,EBITDA_Margin,free_cash_flow,Debt_to_Equity_Score,Net_Profit_Margin_Score,ROE_Score,EBITDA_Margin_Score,free_cash_flow_Score,Total_Score
1,AAPL,0.243528,1.477431,2.161000,0.326608,94336250.00,0.400441,0.877399,0.642225,0.696700,1.000000,0.744125
271,MSFT,0.358393,0.399512,0.342611,0.518689,65776800.00,0.412353,0.912413,0.592420,0.786158,0.725065,0.722743
393,VICI,0.645717,0.095323,0.626061,0.888583,1634472.80,0.410496,1.000000,0.578365,0.958429,0.107583,0.691386
316,PLD,0.472767,0.057283,0.473510,0.921012,3858133.75,0.411495,0.947279,0.576607,0.973532,0.128990,0.684852
176,GOOG,0.246971,0.264708,0.112283,0.342487,60029400.00,0.413862,0.878448,0.586191,0.704095,0.669737,0.684030
175,GOOGL,0.246971,0.264708,0.112283,0.342487,60029400.00,0.413862,0.878448,0.586191,0.704095,0.669737,0.684030
404,V,0.518915,0.401980,0.595186,0.696315,16152200.00,0.410698,0.961346,0.592534,0.868884,0.247342,0.683862
324,PSA,0.542314,0.221734,0.669130,0.853657,2469583.20,0.410214,0.968479,0.584206,0.942163,0.115623,0.682077
274,MTD,0.208506,9.220715,20.705700,0.302978,786108.80,0.278957,0.866722,1.000000,0.685695,0.099416,0.680912
79,CME,0.530422,0.102984,0.128109,0.795912,2925780.00,0.413758,0.964854,0.578719,0.915269,0.120015,0.674088


In [58]:
df_ratio.sort_values(by='Net_Profit_Margin', ascending=False).head(20)

,Company,Net_Profit_Margin,ROE,Debt_to_Equity,EBITDA_Margin,free_cash_flow,Debt_to_Equity_Score,Net_Profit_Margin_Score,ROE_Score,EBITDA_Margin_Score,free_cash_flow_Score,Total_Score
393,VICI,0.645717,0.095323,0.626061,0.888583,1634472.80,0.410496,1.000000,0.578365,0.958429,0.107583,0.691386
398,VRSN,0.563939,-0.539308,-1.248990,0.706606,763172.00,0.422779,0.975071,0.549042,0.873677,0.099196,0.656605
324,PSA,0.542314,0.221734,0.669130,0.853657,2469583.20,0.410214,0.968479,0.584206,0.942163,0.115623,0.682077
79,CME,0.530422,0.102984,0.128109,0.795912,2925780.00,0.413758,0.964854,0.578719,0.915269,0.120015,0.674088
404,V,0.518915,0.401980,0.595186,0.696315,16152200.00,0.410698,0.961346,0.592534,0.868884,0.247342,0.683862
316,PLD,0.472767,0.057283,0.473510,0.921012,3858133.75,0.411495,0.947279,0.576607,0.973532,0.128990,0.684852
126,EBAY,0.470257,0.720390,1.560197,0.266974,1893400.00,0.404376,0.946513,0.607246,0.668927,0.110076,0.612621
248,MA,0.447222,1.421746,2.064245,0.593075,9430200.00,0.401075,0.939492,0.639652,0.820802,0.182631,0.667488
222,KIM,0.420665,0.072902,0.809148,0.834741,816597.40,0.409297,0.931396,0.577329,0.933353,0.099710,0.666406
330,REGN,0.393429,0.273135,0.152900,0.468600,3899625.00,0.413595,0.923094,0.586581,0.762830,0.129389,0.628894


In [59]:
df_ratio.sort_values(by='ROE', ascending=False).head(20)

,Company,Net_Profit_Margin,ROE,Debt_to_Equity,EBITDA_Margin,free_cash_flow,Debt_to_Equity_Score,Net_Profit_Margin_Score,ROE_Score,EBITDA_Margin_Score,free_cash_flow_Score,Total_Score
274,MTD,0.208506,9.220715,20.705700,0.302978,786108.80,0.278957,0.866722,1.000000,0.685695,0.099416,0.680912
183,HD,0.102218,6.740937,22.770545,0.165823,15320200.00,0.265431,0.834322,0.885423,0.621817,0.239332,0.647833
166,GDDY,0.146114,5.853514,-51.951185,0.129162,859940.00,0.754922,0.847703,0.844420,0.604744,0.100127,0.664728
356,STX,0.059013,3.348038,8.169131,0.150098,871600.00,0.361082,0.821152,0.728655,0.614494,0.100239,0.592219
224,KMB,0.099599,2.395608,10.349545,0.173079,2217000.00,0.346799,0.833524,0.684649,0.625197,0.113191,0.587501
298,ORCL,0.213933,2.348284,19.857791,0.392698,10065600.00,0.284512,0.868377,0.682462,0.727480,0.188748,0.626343
77,CL,0.126378,2.324112,9.125294,0.221747,2741750.00,0.354819,0.841687,0.681345,0.647863,0.118243,0.595942
213,IT,0.132404,1.806480,7.435591,0.232669,1029502.75,0.365888,0.843524,0.657428,0.652950,0.101760,0.590328
1,AAPL,0.243528,1.477431,2.161000,0.326608,94336250.00,0.400441,0.877399,0.642225,0.696700,1.000000,0.744125
406,WAT,0.226333,1.429972,3.841929,0.341056,525692.60,0.389429,0.872157,0.640032,0.703429,0.096910,0.607384


In [60]:
df_ratio.sort_values(by='EBITDA_Margin', ascending=False).head(20)

,Company,Net_Profit_Margin,ROE,Debt_to_Equity,EBITDA_Margin,free_cash_flow,Debt_to_Equity_Score,Net_Profit_Margin_Score,ROE_Score,EBITDA_Margin_Score,free_cash_flow_Score,Total_Score
139,ESS,0.303950,0.079547,1.043118,0.977844,805885.40,0.407764,0.895817,0.577636,1.000000,0.099607,0.674081
316,PLD,0.472767,0.057283,0.473510,0.921012,3858133.75,0.411495,0.947279,0.576607,0.973532,0.128990,0.684852
393,VICI,0.645717,0.095323,0.626061,0.888583,1634472.80,0.410496,1.000000,0.578365,0.958429,0.107583,0.691386
302,O,0.212911,0.028175,0.684491,0.877750,2242571.60,0.410113,0.868065,0.575262,0.953383,0.113437,0.657205
352,SPG,0.371582,0.497632,6.790951,0.854712,2801534.00,0.370111,0.916434,0.596953,0.942654,0.118818,0.668844
324,PSA,0.542314,0.221734,0.669130,0.853657,2469583.20,0.410214,0.968479,0.584206,0.942163,0.115623,0.682077
222,KIM,0.420665,0.072902,0.809148,0.834741,816597.40,0.409297,0.931396,0.577329,0.933353,0.099710,0.666406
137,EQR,0.358645,0.083324,0.705125,0.808300,1173199.60,0.409978,0.912490,0.577810,0.921038,0.103143,0.659304
79,CME,0.530422,0.102984,0.128109,0.795912,2925780.00,0.413758,0.964854,0.578719,0.915269,0.120015,0.674088
223,KKR,0.215648,0.050448,0.784777,0.781884,-2495856.60,0.409456,0.868900,0.576291,0.908736,0.067822,0.639601


In [78]:
df = pd.read_csv('top_20_companies_for_investment.csv')

data = pd.read_csv('./stock_dfs2/A.csv')

data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d %H:%M:%S-%z:%z')

data.set_index('Date', inplace=True)

mpf.plot(data, type='candle', volume=True, title='Stock Price Data', style='yahoo')

plt.show()


error: redefinition of group name 'z' as group 10; was group 7 at position 216